# 项目：评估、清洗和整理沃尔玛销售数据

## 1. 简介

### 分析目标

本实战项目的目的是，练习评估数据的结构性问题和内容性问题，并且基于评估结果对沃尔玛的销售数据进行清洗、整理，从而得到干净、整洁的数据，供下一步用于探索不同地区沃尔玛零售店销售额的影响因素，通过进一步创建预测模型，可以预测未来X个月/年的销售情况，并针对零售门店库存管理方面的问题提出建议，以使得供应与需求更加匹配。

#### 数据每列的含义如下：


- `Store`：店铺编号
- `Date`：销售周
- `Weekly_Sales`：店铺在该周的销售额
- `Holiday_Flag`：是否为假日周
- `Temperature`：销售日的温度
- `Fuel_Price`：该地区的燃油成本
- `CPI（消费者物价指数）`：消费者物价指数
- `Unemployment`：失业率

## 2. 读取数据

导入数据分析所需要的库，并通过Pandas的 `read_csv` 函数，将原始文件`walmart_stores_data.csv`里的数据内容，解析为DataFrame并赋值给变量 `original_data`。

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so

np.set_printoptions(suppress=True)
pd.set_option('display.float_format', lambda x: '%.3f' %x)

In [2]:
original_data = pd.read_csv("walmart_stores_data.csv")
original_data.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.900,0,42.310,2.572,211.096,8.106
1,1,12-02-2010,1641957.440,1,38.510,2.548,211.242,8.106
2,1,19-02-2010,1611968.170,0,39.930,2.514,211.289,8.106
3,1,26-02-2010,1409727.590,0,46.630,2.561,211.320,8.106
4,1,05-03-2010,1554806.680,0,46.500,2.625,211.350,8.106


## 3. 评估和清洗数据

在这一部分中，我们将对在上一部分建立的 `original_data` DataFrame所包含的数据进行评估和清理。

主要从两个方面进行：结构和内容，即整齐度和干净度。

数据的结构性问题指不符合“每个变量为一列，每个观察值为一行，每种类型的观察单位为一个表格”这三个标准；数据的内容性问题包括存在丢失数据、重复数据、无效数据等。

为了区分开经过清理的数据和原始的数据，我们创建新的变量 `cleaned_data`，让它为 `original_data` 复制出的副本。我们之后的清理步骤都将被运用在 `cleaned_data` 上。

In [3]:
cleaned_data = original_data.copy()

### 3.1 数据整齐度

In [4]:
cleaned_data.sample(10)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
1925,14,13-05-2011,2004330.300,0,60.380,4.066,186.072,8.521
9,1,09-04-2010,1545418.530,0,65.860,2.770,210.623,7.808
4635,33,25-03-2011,238084.080,0,63.340,3.716,128.616,8.951
5734,41,14-05-2010,1116439.020,0,40.230,2.788,189.490,7.363
5156,37,02-04-2010,540189.700,0,63.430,2.719,209.594,8.464
4319,31,27-08-2010,1326621.980,0,86.200,2.619,211.224,8.099
447,4,11-06-2010,1870619.230,0,78.450,2.668,126.112,7.896
3261,23,20-04-2012,1287899.410,0,56.550,4.046,137.923,4.125
5173,37,30-07-2010,487912.950,0,81.880,2.640,210.129,8.360
3497,25,06-05-2011,659446.550,0,49.500,4.046,208.799,7.287


从抽样的数据来看，数据符合“每列一个变量，每行一个观察值，每个单元格一个值”，具体来看每行是一个沃尔玛销售门店一个周的销售记录，每列是销售相关的各个变量，不存在结构性问题。

### 3.2 数据干净度

接下来通过`info`，对数据内容进行大致了解。

In [5]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


从输出结果来看，`cleaned_data`数据共有6435条观察值,且每个变量均不存在缺失值。

此外，`Store`变量的值所代表的是销售门店，相当于ID，不需要进行代数运算，其数据类型应从整数改为字符串；`Date`表示销售周，数据类型不应为字符串，应为日期，所以需要进行数据格式转换。

In [6]:
cleaned_data['Store'] = cleaned_data['Store'].astype("str")
cleaned_data['Store']

0        1
1        1
2        1
3        1
4        1
        ..
6430    45
6431    45
6432    45
6433    45
6434    45
Name: Store, Length: 6435, dtype: object

In [7]:
cleaned_data['Date'] = pd.to_datetime(cleaned_data['Date'], format='%d-%m-%Y')
cleaned_data['Date']

0      2010-02-05
1      2010-02-12
2      2010-02-19
3      2010-02-26
4      2010-03-05
          ...    
6430   2012-09-28
6431   2012-10-05
6432   2012-10-12
6433   2012-10-19
6434   2012-10-26
Name: Date, Length: 6435, dtype: datetime64[ns]

#### 3.2.1 处理缺失数据

该数据集不存在缺失数据。

#### 3.2.2 处理重复数据

根据数据变量的含义以及内容来看，`cleaned_data`里不应该存在每个变量值都相同的观察值，因此查看是否存在重复值。

In [8]:
cleaned_data.duplicated().sum()

0

输出结果为0，说明不存在重复值。

#### 3.2.3 处理不一致数据

该数据集不含有可能存在不一致数据的变量。

#### 3.2.4 处理无效或错误数据

可以通过DataFrame的`describe`方法，对数值统计信息进行快速了解。

In [9]:
cleaned_data.describe()

,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,6435,6435.000,6435.000,6435.000,6435.000,6435.000,6435.000
mean,2011-06-17 00:00:00,1046964.878,0.070,60.664,3.359,171.578,7.999
min,2010-02-05 00:00:00,209986.250,0.000,-2.060,2.472,126.064,3.879
25%,2010-10-08 00:00:00,553350.105,0.000,47.460,2.933,131.735,6.891
50%,2011-06-17 00:00:00,960746.040,0.000,62.670,3.445,182.617,7.874
75%,2012-02-24 00:00:00,1420158.660,0.000,74.940,3.735,212.743,8.622
max,2012-10-26 00:00:00,3818686.450,1.000,100.140,4.468,227.233,14.313
std,NaN,564366.622,0.255,18.445,0.459,39.357,1.876


从以上统计信息来看，`cleaned_data`里不存在脱离现实意义的数值。  
  
且该销售数据集所选取的是2010-02-05到2012-10-26的销售数据。

## 4. 整理数据

### 4.1 特征观察与处理

In [10]:
cleaned_data

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,2010-02-05,1643690.900,0,42.310,2.572,211.096,8.106
1,1,2010-02-12,1641957.440,1,38.510,2.548,211.242,8.106
2,1,2010-02-19,1611968.170,0,39.930,2.514,211.289,8.106
3,1,2010-02-26,1409727.590,0,46.630,2.561,211.320,8.106
4,1,2010-03-05,1554806.680,0,46.500,2.625,211.350,8.106
...,...,...,...,...,...,...,...,...
6430,45,2012-09-28,713173.950,0,64.880,3.997,192.014,8.684
6431,45,2012-10-05,733455.070,0,64.890,3.985,192.170,8.667
6432,45,2012-10-12,734464.360,0,54.470,4.000,192.327,8.667
6433,45,2012-10-19,718125.530,0,56.470,3.969,192.331,8.667


  
对于后续分析我们想想看一下不同季度的销售额的差异，并对不同季度的销售额进行预测，因此，可以创建一个新的离散型数值变量`Quarter`，分别对应1-3、4-6、7-9、10-12月。

#### 4.1.1 创建季度变量

In [11]:
def numQuarter(row):
    if row.month <= 3:
        return 1
    elif row.month <= 6:
        return 2
    elif row.month <= 9:
        return 3
    else:
        return 4

In [12]:
cleaned_data['Quarter'] = cleaned_data['Date'].apply(numQuarter)
cleaned_data

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter
0,1,2010-02-05,1643690.900,0,42.310,2.572,211.096,8.106,1
1,1,2010-02-12,1641957.440,1,38.510,2.548,211.242,8.106,1
2,1,2010-02-19,1611968.170,0,39.930,2.514,211.289,8.106,1
3,1,2010-02-26,1409727.590,0,46.630,2.561,211.320,8.106,1
4,1,2010-03-05,1554806.680,0,46.500,2.625,211.350,8.106,1
...,...,...,...,...,...,...,...,...,...
6430,45,2012-09-28,713173.950,0,64.880,3.997,192.014,8.684,3
6431,45,2012-10-05,733455.070,0,64.890,3.985,192.170,8.667,4
6432,45,2012-10-12,734464.360,0,54.470,4.000,192.327,8.667,4
6433,45,2012-10-19,718125.530,0,56.470,3.969,192.331,8.667,4


#### 4.1.2 对Temperature进行分箱(离散化)处理

按照四分位点对温度进行分箱，并用分类变量1234表示。

In [13]:
cleaned_data["TemperatureBand"] = pd.qcut(cleaned_data["Temperature"],q=4, labels=[1,2,3,4])
cleaned_data.sample(5)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter,TemperatureBand
1571,11,2012-10-19,1232073.180,0,73.770,3.594,226.969,6.034,4,3
1658,12,2011-09-23,871692.740,0,82.950,3.789,129.518,13.503,3,4
408,3,2012-06-08,446336.800,0,81.650,3.452,225.252,6.664,2,4
1985,14,2012-07-06,1862128.950,0,82.990,3.475,191.134,8.684,3,4
4642,33,2011-05-13,279466.870,0,78.240,4.202,129.089,8.687,2,4


#### 4.1.3 对CPI进行分箱(离散化)处理

按照四分位点对CPI(消费者物价指数)进行分箱，并用分类变量1234表示。

In [14]:
cleaned_data["CPIBand"] = pd.qcut(cleaned_data["CPI"],q=4, labels=[1,2,3,4])
cleaned_data.sample(5)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter,TemperatureBand,CPIBand
1155,9,2010-04-23,488683.570,0,61.260,2.795,213.950,6.384,2,2,4
4310,31,2010-06-25,1303523.730,0,87.010,2.653,210.995,8.200,2,4,3
4037,29,2010-09-24,465338.410,0,65.140,2.718,132.762,10.409,3,3,2
2677,19,2012-01-27,1279623.260,0,32.410,3.737,136.960,7.943,1,1,2
5411,38,2012-05-25,422810.120,0,83.840,4.293,131.029,11.627,2,4,1


#### 4.1.3 对Unemployment进行分箱(离散化)处理

按照四分位点对失业率进行分箱，并用分类变量1234表示。

In [15]:
cleaned_data["UnemploymentBand"] = pd.qcut(cleaned_data["Unemployment"],q=4, labels=[1,2,3,4])
cleaned_data.sample(5)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter,TemperatureBand,CPIBand,UnemploymentBand
3459,25,2010-08-13,686072.390,0,72.730,2.805,204.853,7.527,3,3,3,2
5910,42,2010-12-31,428953.600,1,49.670,3.148,127.088,9.003,4,2,1,4
684,5,2012-03-30,331318.730,0,67.760,3.845,221.948,5.943,1,3,4,1
350,3,2011-04-29,367405.400,0,78.690,3.810,219.024,7.574,2,4,4,2
3305,24,2010-05-28,1473868.150,0,69.590,3.046,132.294,8.211,2,3,2,3


#### 4.1.3 对Fuel_Price进行分箱(离散化)处理

按照四分位点对燃油价进行分箱，并用分类变量1234表示。

In [16]:
cleaned_data["Fuel_PriceBand"] = pd.qcut(cleaned_data["Fuel_Price"],q=4, labels=[1,2,3,4])
cleaned_data.sample(5)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter,TemperatureBand,CPIBand,UnemploymentBand,Fuel_PriceBand
6073,43,2011-05-20,648330.180,0,74.210,3.907,207.520,10.581,2,3,3,4,4
2307,17,2010-06-18,877996.270,0,54.940,2.819,126.114,6.635,2,2,1,1,1
3079,22,2011-07-22,964683.800,0,77.790,3.882,139.797,8.023,3,4,2,3,4
3479,25,2010-12-31,623092.540,1,25.890,3.179,204.643,7.484,4,1,3,2,2
4694,33,2012-05-11,295841.840,0,81.020,4.186,131.145,7.396,2,4,1,2,4


#### 4.1.4 对销售额也进行离散化处理

In [17]:
cleaned_data["WeeklySalesBand"] = pd.qcut(cleaned_data["Weekly_Sales"],q=10, labels=[1,2,3,4,5,6,7,8,9,10])
cleaned_data.sample(5)

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter,TemperatureBand,CPIBand,UnemploymentBand,Fuel_PriceBand,WeeklySalesBand
5714,40,2012-09-21,899768.400,0,54.120,4.056,138.653,4.156,3,2,2,1,4,5
3943,28,2011-09-02,1468871.490,0,93.660,3.798,129.326,13.503,3,4,1,4,4,8
5817,41,2011-12-16,1682368.320,0,25.010,3.266,195.984,6.759,4,1,3,1,2,9
4888,35,2010-08-06,1180183.390,0,77.450,2.784,136.474,8.861,3,4,2,4,1,7
5108,36,2012-01-27,301444.940,0,62.730,3.313,218.769,7.244,1,3,4,2,2,1


#### 保存清理后的数据

In [18]:
cleaned_data.to_csv("cleaned_data.csv")

#### 删除一些列，只保留特征，用于后续机器学习建模

In [20]:
clear_cleaned_data = cleaned_data.drop(labels=['Date','Weekly_Sales','TemperatureBand','CPIBand','UnemploymentBand','Fuel_PriceBand','WeeklySalesBand'],axis=1)
clear_cleaned_data

,Store,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Quarter
0,1,0,42.310,2.572,211.096,8.106,1
1,1,1,38.510,2.548,211.242,8.106,1
2,1,0,39.930,2.514,211.289,8.106,1
3,1,0,46.630,2.561,211.320,8.106,1
4,1,0,46.500,2.625,211.350,8.106,1
...,...,...,...,...,...,...,...
6430,45,0,64.880,3.997,192.014,8.684,3
6431,45,0,64.890,3.985,192.170,8.667,4
6432,45,0,54.470,4.000,192.327,8.667,4
6433,45,0,56.470,3.969,192.331,8.667,4


In [21]:
clear_cleaned_data.to_csv("clear_cleaned_data.csv")